# Enkrypt Secure MCP Gateway - Complete Setup Walkthrough

This notebook provides a comprehensive walkthrough for setting up a complete MCP (Model Context Protocol) configuration using the Enkrypt Secure MCP Gateway. 

## What We'll Cover:

1. **Create a new MCP configuration** - Set up a named configuration
2. **Add a server to the configuration** - Configure an MCP server with commands and arguments
3. **Add input and output guardrails** - Implement security policies for server communications
4. **Create a project** - Organize configurations into manageable projects
5. **Assign MCP config to project** - Link configuration to project
6. **Create a new user** - Set up user accounts for access control
7. **Add user to project** - Grant user access to specific projects
8. **Generate API key** - Create secure access credentials for the user

This workflow represents a complete setup from initial configuration to user access, implementing security best practices with guardrails.


## Prerequisites

Before starting, ensure you have:
- The Enkrypt Secure MCP Gateway CLI installed
- Access to run `secure-mcp-gateway` commands

Some Useful Links:
- Install the Seure MCP Gateway: https://www.youtube.com/watch?v=k-3Hi8cyr-E&list=PLnPPW2f7VGuJlV8u4ViFqyCgsyccKp9Hw&index=4 
- GitHub Docs for the Gateway: https://github.com/enkryptai/secure-mcp-gateway
- Documentation of ALL CLI Commands: https://github.com/enkryptai/secure-mcp-gateway/blob/main/CLI-Commands-Reference.md

Let's begin by checking if we have access to the CLI tool:


In [ ]:
# Check if the CLI tool is available
!secure-mcp-gateway --help | head -10


## Step 1: Create a New MCP Configuration

First, we'll create a new MCP configuration. This configuration will serve as a container for our MCP servers and their settings.

### List existing configurations (optional - to see current state):


In [ ]:
# List all existing configurations
!secure-mcp-gateway config list


### Create our new configuration:

We'll create a configuration named "production-demo-config" for this walkthrough:


In [ ]:
# Create a new MCP configuration
!secure-mcp-gateway config add --config-name "production-demo-config"


### Verify the configuration was created:


In [ ]:
# Get details of our newly created configuration
!secure-mcp-gateway config get --config-name "production-demo-config"


## Step 2: Add Server to Configuration

Now we'll add an MCP server to our configuration. We'll create a server with environment variables and tools configuration.


In [ ]:
# Add a server with basic configuration, environment variables, and tools
!secure-mcp-gateway config add-server \
    --config-name "production-demo-config" \
    --server-name "github_server" \
    --server-command "docker" \
    --args "run -i --rm -e GITHUB_PERSONAL_ACCESS_TOKEN ghcr.io/github/github-mcp-server" \
    --env '{"GITHUB_PERSONAL_ACCESS_TOKEN": "your_github_token"}' \
    --description "GitHub MCP Server"


### Verify the server was added:


In [ ]:
# List servers in our configuration
!secure-mcp-gateway config list-servers --config-name "production-demo-config"


## Step 3: Add Input and Output Guardrails

Guardrails provide security policies for server communications. We'll add both input and output guardrails to protect against malicious content and ensure output quality.

### Add Input Guardrails:
Input guardrails help protect against injection attacks, policy violations, and malicious input:


In [ ]:
# Add input guardrails to our server
!secure-mcp-gateway config update-server-input-guardrails \
    --config-name "production-demo-config" \
    --server-name "github_server" \
    --policy '{"enabled": true, "policy_name": "github_mcp_guardrails", "additional_config": {"pii_redaction": false}, "block": ["policy_violation"]}'
    


### Add Output Guardrails:
Output guardrails ensure response quality and filter harmful content:


In [ ]:
# Add output guardrails to our server
!secure-mcp-gateway config update-server-output-guardrails \
    --config-name "production-demo-config" \
    --server-name "github_server" \
    --policy '{"enabled": true, "policy_name": "github_mcp_guardrails", "additional_config": {"relevancy": false, "hallucination": false, "adherence": false}, "block": ["policy_violation"]}'


### Verify the guardrails were added:

Let's check our server configuration to ensure both guardrails are properly set:


In [ ]:
# Get detailed server information including guardrails
!secure-mcp-gateway config get-server --config-name "production-demo-config" --server-name "github_server"


## Step 4: Create a Project

Projects help organize configurations and manage user access. We'll create a project for our demo environment.


In [ ]:
# Create a new project
!secure-mcp-gateway project create --project-name "Demo Production Environment"


## Step 5: Assign MCP Configuration to Project

Now we'll link our configuration to the project, making the servers available to project users.


In [ ]:
# Assign our configuration to the project
!secure-mcp-gateway project assign-config --project-name "Demo Production Environment" --config-name "production-demo-config"


### Verify the configuration assignment:


In [ ]:
# Get project details to see the assigned configuration
!secure-mcp-gateway project get --project-name "Demo Production Environment"


## Step 6: Create a New User

Users need to be created and assigned to projects to access the MCP servers. We'll create a user for our demo environment.


In [ ]:
# Create a new user
!secure-mcp-gateway user create --email "nitin@enkryptai.com"


## Step 7: Add User to Project

To grant the user access to our project and its associated MCP configuration, we need to add them to the project.


In [ ]:
# Add user to the project
!secure-mcp-gateway project add-user --project-name "Demo Production Environment" --email "nitin@enkryptai.com"


## Step 8: Generate API Key for User

The final step is to generate an API key that the user can use to authenticate and access the MCP servers through the gateway.


In [ ]:
# Generate API key for the user in this project
!secure-mcp-gateway user generate-api-key --email "nitin@enkryptai.com" --project-name "Demo Production Environment"


## Summary

🎉 **Congratulations!** You have successfully completed the full setup of an Enkrypt Secure MCP Gateway environment.

### What We Accomplished:

1. ✅ **Created MCP Configuration**: "production-demo-config"
2. ✅ **Added Server**: "demo-api-server" with environment variables and tools
3. ✅ **Implemented Security**: Both input and output guardrails configured
4. ✅ **Created Project**: "Demo Production Environment"
5. ✅ **Linked Configuration**: Assigned config to project
6. ✅ **Created User**: "demo.user@company.com"
7. ✅ **Granted Access**: Added user to project
8. ✅ **Generated Credentials**: API key created for secure access

### Security Features Enabled:

**Input Guardrails:**
- PII redaction
- Content filtering
- Rate limiting  
- Protection against: policy violations, injection attacks, malicious input, sensitive data

**Output Guardrails:**
- Relevancy checking
- Hallucination detection
- Adherence monitoring
- Toxicity filtering
- Content safety
- Protection against: policy violations, hallucinations, harmful content, toxic content

### Server Configuration:

**Environment Variables:**
- API_KEY, ENVIRONMENT, LOG_LEVEL, PORT

**Tools Configuration:**
- Web search (enabled)
- File system (enabled, read-only)
- Calculator (disabled)

### Next Steps:

1. **Use the API Key**: The user can now use their generated API key to authenticate with the MCP Gateway
2. **Monitor Usage**: Use the CLI to monitor API key usage and system health
3. **Scale Up**: Add more servers, users, or projects as needed
4. **Maintain Security**: Regularly rotate API keys and review guardrail policies

### Useful Commands for Ongoing Management:

```bash
# Monitor API keys
secure-mcp-gateway user list-api-keys --email "demo.user@company.com"

# Rotate API key when needed
secure-mcp-gateway user rotate-api-key --api-key "YOUR_API_KEY"

# Regular health checks
secure-mcp-gateway system health-check

# Create backups
secure-mcp-gateway system backup --output-file "backup-$(date +%Y%m%d).json"
```

This completes our walkthrough of the Enkrypt Secure MCP Gateway setup process!
